## Dados da ANCINE

https://www.gov.br/ancine/pt-br/oca/dados-abertos

In [1]:
import requests
from zipfile import ZipFile
from io import BytesIO

req = requests.get('https://dados.ancine.gov.br/dados-abertos/bilheteria-diaria-obras-por-exibidoras-csv.zip')
zip_files = ZipFile(BytesIO(req.content))

In [2]:
# Parâmetros

ano = '2024'
mes = '08'
semana = 's01'

In [3]:
import pandas as pd

bilheteria = pd.read_csv(zip_files.open(f'bilheteria-diaria-obras-por-exibidoras-{ano}-{mes}-{semana}.csv'), sep=';')

In [12]:
bilheteria_rj = bilheteria[(bilheteria['UF_SALA_COMPLEXO'] == 'RJ') & (bilheteria['MUNICIPIO_SALA_COMPLEXO'] == 'RIO DE JANEIRO')]
bilheteria_rj.to_csv('../data/bilheteria_rj.csv')

In [56]:
url = 'https://dados.ancine.gov.br/dados-abertos/salas-de-exibicao-e-complexos.csv'

salas = pd.read_csv(url, sep=';')

In [57]:
salas_rj = salas[(salas['UF_COMPLEXO'] == 'RJ') & (salas['MUNICIPIO_COMPLEXO'] == 'RIO DE JANEIRO') & (salas['SITUACAO_SALA'] == 'EM FUNCIONAMENTO')]
salas_rj = salas_rj.drop([1866, 1867, 1868, 1869, 1870])
salas_rj.to_csv('../data/salas_rj_ancine.csv')

## Determinando as RAs a partir dos polígonos

In [ ]:
from shapely.geometry import Point

geometry = [Point(xy) for xy in zip(ingresso['longitude'], ingresso['latitude'])]
gdf_cinemas = gpd.GeoDataFrame(ingresso, geometry=geometry)

In [ ]:
gdf_ra = shapefile_rj.to_crs(epsg=4326)
gdf_cinemas = gdf_cinemas.set_crs(epsg=4326)

In [ ]:
gdf_resultado = gpd.sjoin(gdf_cinemas, gdf_ra, how="left", predicate="within")

In [ ]:
gdf_resultado[['name', 'neighborhood', 'regiao_adm']]

,name,neighborhood,regiao_adm
0,Cine Araújo Jardim Guadalupe,Guadalupe,ANCHIETA
1,Cine Odeon - Centro Cultural Luiz Severiano Ri...,Centro,CENTRO
2,Cine Santa Teresa,Santa Teresa,SANTA TEREZA
3,CineCarioca Méier,Méier,MEIER
4,Cineflix Center Shopping Rio de Janeiro (RIO),Tanque,JACAREPAGUA
5,Cinemark Botafogo,Botafogo,BOTAFOGO
6,Cinemark Carioca Shopping,Vila Kosmos,IRAJA
7,Cinemark Downtown,Barra da Tijuca,BARRA DA TIJUCA
8,Cinemark Metropolitano Barra,Barra da Tijuca,JACAREPAGUA
9,Cinemark Village Mall,Barra da Tijuca,BARRA DA TIJUCA


## Testando outras cores por RA

In [ ]:
def gerar_cor(nome_ra):
    cores = {
            'BOTAFOGO': '#ff0000',
            'BARRA DA TIJUCA': '#0000ff',
            'MEIER': '#00ff00',
            'JACAREPAGUA': '#ffa500',
        }
    return cores.get(nome_ra, '#808080')

import folium

lat = -22.9068
lon = -43.1729

mapa_rj = folium.Map(location=[lat, lon], zoom_start=11)

folium.GeoJson(
    gdf_ra,
    style_function=lambda feature: {
        'fillColor': gerar_cor(feature['properties']['regiao_adm']),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.5,
    },
    tooltip=folium.GeoJsonTooltip(fields=['regiao_adm'], aliases=['Região Administrativa:']),
).add_to(mapa_rj)

folium.LayerControl().add_to(mapa_rj)